In [4]:
import os
import pandas as pd
from datasets import Dataset, Audio
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import get_peft_model, LoraConfig, TaskType
import torch
from dataclasses import dataclass
from typing import Dict, List, Union
import evaluate
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer
from typing import Any, Dict, List, Union

os.chdir(r"D:\Git_repos\Persian_text_correction\datasets")

In [5]:
# Model & Processor

# model_name = "openai/whisper-medium"
model_name = "openai/whisper-small"
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name)

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

D:\Projects\venv2\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
df = pd.read_excel(r"D:\Git_repos\Persian_text_correction\datasets\myaudio_tiny\myaudio_tiny.xlsx")
df = df.rename(columns={"audio": "audio_path", "text": "transcription"})

print(df.columns) 

df.head

Index(['audio_path', 'transcription'], dtype='object')


<bound method NDFrame.head of                 audio_path                                      transcription
0       myaudio/450001.wav  به نام خدا درسی رو که در خدمت شما خواهیم بود آ...
1       myaudio/450002.wav  در واقع کار کردن با لتک و ایجاد مستندات و متن ...
2       myaudio/450003.wav  انگلیسی و کلا مستندات علمی هدف این است که تا ج...
3       myaudio/450004.wav  که ممکنه مطالبی رو که بیشتر کارایی داره براتون...
4       myaudio/450005.wav  آموزش داده بشه و یه مجموعه ی خوبی در اختیار شم...
...                    ...                                                ...
1077  myaudio/12560463.wav  پارک جنگلی چیتگر از پارک های جنگلی شهر تهران ا...
1078  myaudio/12560465.wav  از بزرگترین بوستان های جنگلی استان تهران است ه...
1079  myaudio/12560466.wav  مسیر وزیدن باد غربی به شهر تهران قرار گرفته با...
1080  myaudio/12560467.wav  بوستان جنگلی کوهسار در شمال غربی شهر تهران و د...
1081  myaudio/12560468.wav  پارک جنگلی وردآورد یکی از پارک های جنگلی استان...

[1082 rows x 2 columns]>

In [76]:
# from transformers import Seq2SeqTrainer

# class WhisperSeq2SeqTrainer(Seq2SeqTrainer):
#     def prepare_inputs(self, inputs):
#         # Replace "input_features" instead of "input_ids"
#         if "input_features" in inputs:
#             inputs["input_features"] = inputs.pop("input_features")
#         return super().prepare_inputs(inputs)

In [77]:
# def prepare_example(batch):
#     audio = batch["audio_path"]
#     inputs = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt")
#     input_features = inputs.input_features[0]
#     labels = processor.tokenizer(batch["transcription"], return_tensors="pt").input_ids[0]
#     return {"input_features": input_features, "labels": labels}

# os.chdir(r"D:\Git_repos\Persian_text_correction\datasets\myaudio_tiny")
# dataset = Dataset.from_pandas(df)
# dataset = dataset.cast_column("audio_path", Audio())
# dataset = dataset.map(prepare_example)

In [7]:
def prepare_batch(batch):
    audio = batch["audio_path"]  # <- Fix here
    inputs = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt")

    input_features = inputs["input_features"][0]
    labels = processor.tokenizer(batch["transcription"], return_tensors="pt").input_ids[0]

    return {
        "input_features": input_features,
        "labels": labels
    }

os.chdir(r"D:\Git_repos\Persian_text_correction\datasets\myaudio_tiny")

dataset = Dataset.from_pandas(df)
dataset = dataset.cast_column("audio_path", Audio())
dataset = dataset.map(prepare_batch)

Map:   0%|          | 0/1082 [00:00<?, ? examples/s]

In [82]:
dataset.features

{'audio_path': Audio(sampling_rate=None, mono=True, decode=True, id=None),
 'transcription': Value(dtype='string', id=None),
 'input_features': Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [ ]:
n = 8
print(len(dataset[n]["labels"]))
print(dataset[n]["transcription"])

43
نداره یعنی کسی از فروش نرم افزار حق کسب درآمد نداریم لتک


In [ ]:
dataset[100]["input_features"]

[[0.3319051265716553,
  0.2651655077934265,
  0.5290313363075256,
  0.463532030582428,
  0.7084238529205322,
  0.6327102184295654,
  0.5828836560249329,
  0.54378342628479,
  0.14061641693115234,
  0.5979247093200684,
  -0.03137660026550293,
  0.38390517234802246,
  0.42508167028427124,
  0.23712527751922607,
  0.6773443222045898,
  0.5431109666824341,
  0.6298059225082397,
  0.5348333716392517,
  0.5053048133850098,
  0.6466584801673889,
  0.574289083480835,
  0.6548007726669312,
  0.6372396945953369,
  0.6182392835617065,
  0.7363579273223877,
  0.7833728194236755,
  0.682112455368042,
  0.6218010187149048,
  0.5048904418945312,
  0.5783795714378357,
  0.6053606271743774,
  0.3322448134422302,
  0.5567818880081177,
  0.386854350566864,
  0.7767540216445923,
  0.5154640674591064,
  0.6708282232284546,
  0.3614500164985657,
  0.4590059518814087,
  0.6230399012565613,
  0.7235212326049805,
  0.7763721942901611,
  0.7929725646972656,
  0.834111750125885,
  0.8150376677513123,
  0.8624771

In [ ]:
sample = dataset[5]["audio_path"]["array"].shape
sample

(129444,)

In [ ]:
dataset[0].keys()

dict_keys(['audio_path', 'transcription', 'input_features', 'labels'])

In [ ]:
dataset.features

{'audio_path': Audio(sampling_rate=None, mono=True, decode=True, id=None),
 'transcription': Value(dtype='string', id=None),
 'input_features': Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [ ]:
print(len(dataset[0]["input_features"][0]))
print(len(dataset[0]["input_features"]))

3000
80


In [9]:
# LoRA setup
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM,
)
model = get_peft_model(model, lora_config)

In [10]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item(): labels = labels[:, 1:]

        batch["labels"] = labels
        return batch


In [11]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=processor.tokenizer.bos_token_id
)


In [12]:
data_collator

DataCollatorSpeechSeq2SeqWithPadding(processor=WhisperProcessor:
- feature_extractor: WhisperFeatureExtractor {
  "chunk_length": 30,
  "feature_extractor_type": "WhisperFeatureExtractor",
  "feature_size": 80,
  "hop_length": 160,
  "n_fft": 400,
  "n_samples": 480000,
  "nb_max_frames": 3000,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "WhisperProcessor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}

- tokenizer: WhisperTokenizer(name_or_path='openai/whisper-small', vocab_size=50258, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|endoftext|>', '<|startoftranscript|>', '<|en|>', '<|zh|>', '<|de|>', '<|es|>', '<|ru|>', '<|ko|>', '<|fr|>', '<|ja|>', '<|pt|>', '<|tr|>', '<|pl|>', '<|ca|>', '<|nl|>', '<|ar|>', '<|sv|>', '<|it|>', '<|id|>', '<

In [13]:
metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

In [14]:
# Training args
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-fa-finetuned",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    max_steps=200,
    logging_steps=10,
    fp16=True,
    evaluation_strategy="no",
    report_to="none",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
    tokenizer=processor.tokenizer,  # ✅
)

trainer.train()

KeyError: 'input_features'

In [ ]:
# Train
trainer.train()

KeyError: 'input_features'

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("./whisper-finetuned-fa")
